In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import psycopg2
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# Configuración visual
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Cargar variables de entorno
load_dotenv()

# Conexión a base de datos
engine = create_engine(
    f'postgresql://{os.getenv("DB_USER")}:{os.getenv("DB_PASSWORD")}@'
    f'{os.getenv("DB_HOST")}:{os.getenv("DB_PORT")}/{os.getenv("DB_NAME")}'
)

print(" Configuración completada")

 Configuración completada


In [2]:
query_transacciones = """
SELECT * FROM transacciones
WHERE timestamp_inicio >= '2024-07-01'
"""

df_transacciones = pd.read_sql_query(query_transacciones, engine)
df_usuarios = pd.read_sql_query("SELECT * FROM usuarios", engine)
df_metricas = pd.read_sql_query("SELECT * FROM metricas_operativas", engine)

print(f"Transacciones cargadas: {len(df_transacciones):,}")
print(f"Usuarios cargados: {len(df_usuarios):,}")
print(f"Métricas cargadas: {len(df_metricas):,}")

Transacciones cargadas: 100,000
Usuarios cargados: 10,000
Métricas cargadas: 5,054


In [3]:
#  Preparar datos para análisis
# Extraer información temporal
df_transacciones['fecha'] = pd.to_datetime(df_transacciones['timestamp_inicio']).dt.date
df_transacciones['hora'] = pd.to_datetime(df_transacciones['timestamp_inicio']).dt.hour
df_transacciones['dia_semana'] = pd.to_datetime(df_transacciones['timestamp_inicio']).dt.dayofweek
df_transacciones['mes'] = pd.to_datetime(df_transacciones['timestamp_inicio']).dt.month

# Clasificar periodos
df_transacciones['periodo'] = df_transacciones['hora'].apply(
    lambda x: 'Hora Pico' if 18 <= x <= 23 else 'Hora Normal'
)

# Clasificar montos
df_transacciones['rango_monto'] = pd.cut(
    df_transacciones['monto_usd'],
    bins=[0, 100, 1000, 5000, 50000],
    labels=['Bajo (<$100)', 'Medio ($100-$1K)', 'Alto ($1K-$5K)', 'Muy Alto (>$5K)']
)

print(" Datos preparados para análisis")

 Datos preparados para análisis
